These are the tutorials, some give flask routes, others save htmls

See [tutorials](https://github.com/bokeh/bokeh/tree/3.3.0/examples/output/apis) 

In [ ]:
import json

from flask import Flask
from jinja2 import Template

from bokeh.embed import json_item
from bokeh.plotting import figure
from bokeh.resources import CDN
from bokeh.sampledata.iris import flowers

from bokeh.models import ColumnDataSource, HoverTool, TapTool, CustomJS, Div
from bokeh.layouts import column
from bokeh.themes import Theme

app = Flask(__name__)

page = Template("""
<!DOCTYPE html>
<html lang="en">
<head>
  {{ resources }}
</head>

<body>
  <div id="myplot"></div>
  <div id="myplot2"></div>
  <script>
  fetch('/plot')
    .then(function(response) { return response.json(); })
    .then(function(item) { return Bokeh.embed.embed_item(item); })
  </script>
</body>
""")

colormap = {'setosa': 'red', 'versicolor': 'green', 'virginica': 'blue'}
colors = [colormap[x] for x in flowers['species']]

def make_plot(x, y):
    p = figure(title = "Iris Morphology", sizing_mode="fixed", width=400, height=400)
    p.xaxis.axis_label = x
    p.yaxis.axis_label = y
    p.circle(flowers[x], flowers[y], color=colors, fill_alpha=0.2, size=10)
    
    hover = HoverTool()
    hover.tooltips = [ ("index", "$index"),
                      ("(x,y)", "($x, $y)"),
                      ]
    
    p.add_tools(hover)
    
    description_div = Div(text="hello", width=1200, height=50)

    layout = column(p, description_div)
    
    return layout

@app.route('/')
def root():
    return page.render(resources=CDN.render())


gray_theme = Theme(json={'attrs':{'Plot':{'background_fill_color':'#eeeeee'}}})

@app.route('/plot')
def plot():
    p = make_plot('petal_width', 'petal_length')
    return json.dumps(json_item(p, "myplot", theme=gray_theme))

if __name__ == '__main__':
    app.run()

In [ ]:
from jinja2 import Template

from bokeh.embed import components
from bokeh.plotting import figure
from bokeh.resources import INLINE
from bokeh.sampledata.iris import flowers
from bokeh.themes import Theme
from bokeh.transform import factor_cmap, factor_mark
from bokeh.util.browser import view

SPECIES = ['setosa', 'versicolor', 'virginica']
MARKERS = ['hex', 'circle_x', 'triangle']

p = figure(title = "Iris Morphology")
p.xaxis.axis_label = 'Petal Length'
p.yaxis.axis_label = 'Sepal Width'

p.scatter("petal_length", "sepal_width", source=flowers, legend_group="species", fill_alpha=0.4, size=12,
          marker=factor_mark('species', MARKERS, SPECIES),
          color=factor_cmap('species', 'Category10_3', SPECIES))

p.legend.background_fill_color = "#3f3f3f"

theme = Theme(json={
    'attrs': {
        'Plot': {
            'background_fill_color': '#3f3f3f',
            'border_fill_color': '#3f3f3f',
            'outline_line_color': '#444444',
            },
        'Axis': {
            'axis_line_color': "white",
            'axis_label_text_color': "white",
            'major_label_text_color': "white",
            'major_tick_line_color': "white",
            'minor_tick_line_color': "white",
            },
        'Legend': {
            'background_fill_color': '#3f3f3f',
            'label_text_color': "white",
        },
        'Grid': {
            'grid_line_dash': [6, 4],
            'grid_line_alpha': .3,
            },
        'Title': {
            'text_color': "white",
            },
        },
    })

script, div = components(p, theme=theme)

template = Template('''<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="utf-8">
        <title>Bokeh Scatter Plots</title>
        {{ resources }}
        {{ script }}
        <style>
            body {
                background: #3f3f3f;
            }

            .embed-wrapper {
                width: 50%;
                height: 400px;
                margin: auto;
            }
        </style>
    </head>
    <body>
        <div class="embed-wrapper">
        {{ div }}
        </div>
    </body>
</html>
''')

resources = INLINE.render()

filename = 'embed_themed.html'

html = template.render(resources=resources,
                       script=script,
                       div=div)

with open(filename, mode="w", encoding="utf-8") as f:
    f.write(html)

view(filename)

In [1]:
from jinja2 import Template

from bokeh.embed import components
from bokeh.models import Range1d
from bokeh.plotting import figure
from bokeh.resources import INLINE
from bokeh.util.browser import view

# create some data
x1 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12]
y1 = [0, 8, 2, 4, 6, 9, 5, 6, 25, 28, 4, 7]
x2 = [2, 5, 7, 15, 18, 19, 25, 28, 9, 10, 4]
y2 = [2, 4, 6, 9, 15, 18, 0, 8, 2, 25, 28]
x3 = [0, 1, 0, 8, 2, 4, 6, 9, 7, 8, 9]
y3 = [0, 8, 4, 6, 9, 15, 18, 19, 19, 25, 28]

# select the tools we want
TOOLS="pan,wheel_zoom,box_zoom,reset,save"

# the red and blue graphs will share this data range
xr1 = Range1d(start=0, end=30)
yr1 = Range1d(start=0, end=30)

# only the green will use this data range
xr2 = Range1d(start=0, end=30)
yr2 = Range1d(start=0, end=30)

# build our figures
p1 = figure(x_range=xr1, y_range=yr1, tools=TOOLS, width=300, height=300)
p1.scatter(x1, y1, size=12, color="red", alpha=0.5)

p2 = figure(x_range=xr1, y_range=yr1, tools=TOOLS, width=300, height=300)
p2.scatter(x2, y2, size=12, color="blue", alpha=0.5)

p3 = figure(x_range=xr2, y_range=yr2, tools=TOOLS, width=300, height=300)
p3.scatter(x3, y3, size=12, color="green", alpha=0.5)

# plots can be a single Bokeh model, a list/tuple, or even a dictionary
plots = dict(Red=p1, Blue=p2, Green=p3)

script, div = components(plots)

template = Template('''<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="utf-8">
        <title>Bokeh Scatter Plots</title>
        {{ resources }}
        {{ script }}
        <style>
            .embed-wrapper {
                display: flex;
                justify-content: space-evenly;
            }
        </style>
    </head>
    <body>
        <div class="embed-wrapper">
            {% for key in div.keys() %}
                {{ div[key] }}
            {% endfor %}
        </div>
    </body>
</html>
''')

resources = INLINE.render()

filename = 'embed_multiple.html'

html = template.render(resources=resources,
                       script=script,
                       div=div)

with open(filename, mode="w", encoding="utf-8") as f:
    f.write(html)

view(filename)

A theme and better tooltips

In [ ]:
import yaml

from bokeh.core.properties import value
from bokeh.document import Document
from bokeh.models import (CDSView, Circle, ColumnDataSource, DataRange1d,
                          Grid, HoverTool, IndexFilter, LinearAxis, Plot,
                          Range1d, SingleIntervalTicker, Text)
from bokeh.sampledata.sprint import sprint
from bokeh.themes import Theme

# Based on http://www.nytimes.com/interactive/2012/08/05/sports/olympics/the-100-meter-dash-one-race-every-medalist-ever.html

abbrev_to_country = {
    "USA": "United States",
    "GBR": "Britain",
    "JAM": "Jamaica",
    "CAN": "Canada",
    "TRI": "Trinidad and Tobago",
    "AUS": "Australia",
    "GER": "Germany",
    "CUB": "Cuba",
    "NAM": "Namibia",
    "URS": "Soviet Union",
    "BAR": "Barbados",
    "BUL": "Bulgaria",
    "HUN": "Hungary",
    "NED": "Netherlands",
    "NZL": "New Zealand",
    "PAN": "Panama",
    "POR": "Portugal",
    "RSA": "South Africa",
    "EUA": "United Team of Germany",
}

fill_color = { "gold": "#efcf6d", "silver": "#cccccc", "bronze": "#c59e8a" }
line_color = { "gold": "#c8a850", "silver": "#b0b0b1", "bronze": "#98715d" }

def selected_name(name, medal, year):
    return name if medal == "gold" and year in [1988, 1968, 1936, 1896] else ""

t0 = sprint.Time[0]

sprint["Abbrev"]       = sprint.Country
sprint["Country"]      = sprint.Abbrev.map(lambda abbr: abbrev_to_country[abbr])
sprint["Medal"]        = sprint.Medal.map(lambda medal: medal.lower())
sprint["Speed"]        = 100.0/sprint.Time
sprint["MetersBack"]   = 100.0*(1.0 - t0/sprint.Time)
sprint["MedalFill"]    = sprint.Medal.map(lambda medal: fill_color[medal])
sprint["MedalLine"]    = sprint.Medal.map(lambda medal: line_color[medal])
sprint["SelectedName"] = sprint[["Name", "Medal", "Year"]].apply(tuple, axis=1).map(lambda args: selected_name(*args))

source = ColumnDataSource(sprint)

xdr =Range1d(start=25, end=-0.5)
ydr = DataRange1d(range_padding=2, range_padding_units="absolute")

plot = Plot(width=1000, x_range=xdr, y_range=ydr, toolbar_location=None)
plot.title.text = "Usain Bolt vs. 116 years of Olympic sprinters"

xticker = SingleIntervalTicker(interval=5, num_minor_ticks=0)
xaxis = LinearAxis(ticker=xticker, axis_label="Meters behind 2012 Bolt")
plot.add_layout(xaxis, "below")

xgrid = Grid(dimension=0, ticker=xaxis.ticker)
plot.add_layout(xgrid)

yticker = SingleIntervalTicker(interval=12, num_minor_ticks=0)
yaxis = LinearAxis(ticker=yticker, major_tick_in=-5, major_tick_out=10)
plot.add_layout(yaxis, "right")

medal = Circle(x="MetersBack", y="Year", size=10, fill_color="MedalFill", line_color="MedalLine", fill_alpha=0.5)
medal_renderer = plot.add_glyph(source, medal)

#sprint[sprint.Medal=="gold" * sprint.Year in [1988, 1968, 1936, 1896]]
view = CDSView(filter=IndexFilter(list(sprint.query('Medal == "gold" and Year in [1988, 1968, 1936, 1896]').index)))
plot.add_glyph(source, Text(x="MetersBack", y="Year", x_offset=10, text="Name"), view=view)

plot.add_glyph(source, Text(x=7.5, y=1942, text=value("No Olympics in 1940 or 1944"),
                            text_font_style="italic", text_color="silver"))

tooltips = """
<div>
    <span style="font-size: 15px;">@Name</span>&nbsp;
    <span style="font-size: 10px; color: #666;">(@Abbrev)</span>
</div>
<div>
    <span style="font-size: 17px; font-weight: bold;">@Time{0.00}</span>&nbsp;
    <span style="font-size: 10px; color: #666;">@Year</span>
</div>
<div style="font-size: 11px; color: #666;">@{MetersBack}{0.00} meters behind</div>
"""

hover = HoverTool(tooltips=tooltips, renderers=[medal_renderer])
plot.add_tools(hover)

theme = Theme(json=yaml.safe_load("""
attrs:
    Plot:
        outline_line_color: !!null
    Axis:
        axis_line_color: !!null
        major_tick_line_color: !!null
        axis_label_text_font_style: "bold"
        axis_label_text_font_size: "10pt"
    Grid:
        grid_line_dash: "dashed"
    Text:
        text_baseline: "middle"
        text_align: "center"
        text_font_size: "9pt"
"""))

doc = Document(theme=theme)
doc.add_root(plot)

if __name__ == "__main__":
    from bokeh.embed import file_html
    from bokeh.util.browser import view

    doc.validate()
    filename = "file_html.html"
    with open(filename, "w") as f:
        f.write(file_html(doc, title=plot.title.text))
    print(f"Wrote {filename}")
    view(filename)